In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
trees_8b = ObjIter([Tree(f) for f in fc.eightb.accstudies.raw_no_pu.signal_list])

100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


In [10]:
decayChain_8b = {
    "X":{
        "Y1": {
            "H1Y1":{
                "H1Y1_b1":{},
                "H1Y1_b2":{},
            },
            "H2Y1":{
                "H2Y1_b1":{},
                "H2Y1_b2":{},
            }
        },
        "Y2": {
            "H1Y2":{
                "H1Y2_b1":{},
                "H1Y2_b2":{},
            },
            "H2Y2":{
                "H2Y2_b1":{},
                "H2Y2_b2":{},
            }
        }
    }
}

In [4]:
from utils.fileUtils.passthrough import NMSSM_List as signal6b_list

trees_6b = ObjIter([Tree(f) for f in signal6b_list[::3]])

100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


In [8]:
decayChain_6b = {
    "X": {
        "Y": {
            "HY1": {
                "HY1_b1": {},
                "HY1_b2": {},
            },
            "HY2": {
                "HY2_b1": {},
                "HY2_b2": {},
            }
        },
        "HX": {
            "HX_b1": {},
            "HX_b2": {},
        }
    }
}


In [11]:
def make_datacard(tree, mask, decayChain=decayChain_8b, nb=8):
    event = tree[ ak.where( mask(tree) )[0][0] ]

    f = open(f'gen_{tree.sample}_{nb}b_datacard.json','w')

    particles = []
    def print_obj(obj, decays=None):

        kins = [ print_obj(obj,_decays) for obj, _decays in decays.items() ]

        if not any(decays):
            p4 = build_p4(event, f'gen_{obj}')
        else:
            p4 = kins[0] + kins[1]

        info =  [f'"name":"{obj}"'] + \
                [ f'"{var}":{getattr(p4,var)}' for var in ('pt','eta','phi','m') ] + \
                [ f'"decays":[' + ','.join([f'"{key}"' for key in decays.keys()]) + ']' ]
        particles.append('{\n' + ',\n'.join(info) + '\n}')
        return p4

    for obj, decays in decayChain.items():
        print_obj('X', decays)

    f.write('{\n"particles":[')
    f.write(',\n'.join(particles))
    f.write(']}\n')

    f.close()

In [12]:
trees_8b.apply(lambda t : make_datacard(t, lambda t : t.nfound_all == 8, decayChain=decayChain_8b, nb=8))

[None, None, None]

In [13]:
trees_6b.apply(lambda t : make_datacard(t, lambda t : t.nfound_all == 6, decayChain=decayChain_6b, nb=6))

[None, None, None]